In [31]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import numpy as np
from scipy.stats import zscore
from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.vector_ar.vecm import VECM, coint_johansen, select_order
from statsmodels.tsa.stattools import grangercausalitytests
import statsmodels.api as sm
import warnings
from statsmodels.tsa.stattools import adfuller
warnings.filterwarnings("ignore")

In [32]:
df = pd.read_csv("프로젝트데이터_외국인보유수_포함.csv", parse_dates=['Date'], index_col='Date')
df = df.drop("CPI_ratio", axis=1)
# df['spread'] = df['call_r'] - df['us_tb_3m']
# df = df[['call_r', 'us_tb_3m']]
df

,kospi,kosdaq,usdkrw,us_tb_3m,call_r,PPI,CPI,IPI,EPI,TOTAL_foreign_own,KOSPI_foreign_own,KOSDAQ_foreign_own
Date,,,,,,,,,,,,
2001-01-02,520.95,557.00,1278.00,5.87,5.26,76.37,64.60,76.45,125.93,22.6,30.4,2.8
2001-01-03,521.43,569.00,1270.50,5.69,5.26,76.37,64.60,76.45,125.93,22.6,30.4,2.8
2001-01-04,558.02,615.10,1263.30,5.37,5.28,76.37,64.60,76.45,125.93,23.0,30.6,3.0
2001-01-05,580.85,625.20,1262.50,5.12,5.31,76.37,64.60,76.45,125.93,23.3,30.8,3.1
2001-01-08,586.65,650.00,1263.30,5.19,5.30,76.37,64.60,76.45,125.93,23.4,30.9,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-04,2662.10,845.84,1372.93,5.52,3.48,119.12,114.09,141.58,131.70,30.6,34.7,9.2
2024-06-05,2689.50,850.75,1368.48,5.51,3.50,119.12,114.09,141.58,131.70,30.7,34.8,9.3
2024-06-07,2722.67,866.18,1378.80,5.52,3.59,119.12,114.09,141.58,131.70,30.7,34.8,9.3


## ADF 테스트

In [60]:
def adfuller_result(cols_data):
    result = adfuller(cols_data)
    return result[1]
    # adf_table = pd.DataFrame(adfuller(df)[:2])
    # adf_table.index = ['ADF Score', 'p-value']
    # return adf_table

In [34]:
# adfuller_result(df['call_r'])

In [67]:
# adfuller_result(df['us_tb_3m'])

,0
ADF Score,-1.338317
p-value,0.611465


### 비정상 시계열 확인

In [81]:
# lag_order = vecm.select_order(data=df, maxlags=22, deterministic="ci")
# lag_order = lag_order.bic
# lag_order

4

## BIC 기준으로 선택
## 이유: BIC는 모델이 간결하게 유지되도록 도와주어 과적합의 위험을 줄이고 인과관계 검정의 신뢰성을 높일 수 있다.

In [69]:
# vec_rank1 = vecm.select_coint_rank(df, det_order = 1, k_ar_diff=lag_order-1, method='trace', signif=0.05)
# print(vec_rank1.summary())

Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   2          20.54          18.40
  1   2          3.047          3.841
-------------------------------------


In [70]:
# 공적분이 하나도 없다라는 귀무 가설에서는 테스트 통계량이 임계점 보다 높기 때문에 귀무가설을 기각할 수 있다.
# 하지만 한개 이상이 있다라는 것을 놓고 봤을 때는 테스트 통계량이 임계점 보다 낮기 때문에 귀무가설을 따라야 한다.
# 즉 공적분 관계가 존재한다는 것을 의미

In [71]:
# vec_rank2 = vecm.select_coint_rank(df, det_order = 1, k_ar_diff=lag_order-1, method='maxeig', signif=0.05)
# print(vec_rank2.summary())

Johansen cointegration test using maximum eigenvalue test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   1          17.49          17.15
  1   2          3.047          3.841
-------------------------------------


In [72]:
# 즉 공적분 관계가 존재한다는 것을 의미

In [73]:
# vecM = VECM(endog=df, k_ar_diff=lag_order-1, coint_rank=1, deterministic='ci')
# vecm_fit = vecM.fit()
# print(vecm_fit.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation call_r
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
L1.call_r      -0.1407      0.013    -10.763      0.000      -0.166      -0.115
L1.us_tb_3m     0.0048      0.014      0.336      0.737      -0.023       0.033
L2.call_r      -0.1194      0.013     -9.100      0.000      -0.145      -0.094
L2.us_tb_3m    -0.0117      0.014     -0.812      0.417      -0.040       0.017
L3.call_r      -0.0849      0.013     -6.484      0.000      -0.111      -0.059
L3.us_tb_3m     0.0462      0.014      3.238      0.001       0.018       0.074
Det. terms outside the coint. relation & lagged endog. parameters for equation us_tb_3m
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
L1.call_r       0.0315    

## 국내 금리 상승

In [35]:
def find_lag_order(df):
    lag_order = vecm.select_order(data=df, maxlags=22, deterministic="ci")
    results = lag_order.summary()
    print(results)
    results_as_html = results.as_html()
    result = pd.read_html(results_as_html, header=0, index_col=0)[0]
    contains_asterisk = result['BIC'].str.contains('\*')
    asterisk_index = result[contains_asterisk].index[0]
    lag_order = asterisk_index
    print("lag order: ", lag_order)
    # if asterisk_index == 0:
    #     result['BIC_Clean'] = pd.to_numeric(result['BIC'].str.replace('*','',regex=True), errors='coerce')
    #     next_min_index = result['BIC_Clean'][1:].idxmin()  # start from index 1 to skip the first
    #     next_min_value = result['BIC_Clean'][next_min_index]
    #     lag_order = next_min_index
    #     print("Index 0 has the asterisk. The next smallest BIC is at index:", next_min_index)
    #     print("BIC value at next smallest index:", next_min_value)
    # if next_min_index == 1:
    #     result['BIC_Clean'] = pd.to_numeric(result['BIC'].str.replace('*','',regex=True), errors='coerce')
    #     next_min_index = result['BIC_Clean'][2:].idxmin()  # start from index 2 to skip the first
    #     next_min_value = result['BIC_Clean'][next_min_index]
    #     lag_order = next_min_index
    #     print("Index 0 has the asterisk. The next smallest BIC is at index:", next_min_index)
    #     print("BIC value at next smallest index:", next_min_value)
    # else:
    #     lag_order = asterisk_index
    #     print("Index of the BIC with asterisk:", asterisk_index)
    #     print("BIC value at asterisk index:", result['BIC'][asterisk_index])
    return lag_order

In [36]:
df = pd.read_csv("프로젝트데이터_외국인보유수_포함.csv", parse_dates=['Date'], index_col='Date')
df = df.drop("CPI_ratio", axis=1)
df = df[['kospi','us_tb_3m']]
# df['spread'] = df['call_r'] - df['us_tb_3m']
# df = df[['call_r', 'us_tb_3m']]
df

,kospi,us_tb_3m
Date,,
2001-01-02,520.95,5.87
2001-01-03,521.43,5.69
2001-01-04,558.02,5.37
2001-01-05,580.85,5.12
2001-01-08,586.65,5.19
...,...,...
2024-06-04,2662.10,5.52
2024-06-05,2689.50,5.51
2024-06-07,2722.67,5.52


In [64]:
def all_test(df):
    sf = df
    print(sf.head())
    print("ADF TEST")
    # print("ADF TEST for call rate")
    a = adfuller_result(sf['kospi'])
    # print(adfuller_result(sf['kospi']))
    print(f"ADF Test result of kospi: {a}")
    b = adfuller_result(sf['us_tb_3m'])
    # print("ADF TEST for us rate")
    # print(adfuller_result(sf['us_tb_3m']))
    print(f"ADF Test result of us_tb_3m: {b}")
    print("Find Lag")
    lag_order = find_lag_order(sf)
    vec_rank1 = vecm.select_coint_rank(sf, det_order = 1, k_ar_diff=lag_order, method='trace', signif=0.05)
    print(vec_rank1.summary())
    vec_rank2 = vecm.select_coint_rank(sf, det_order = 1, k_ar_diff=lag_order, method='maxeig', signif=0.05)
    print(vec_rank2.summary())
    vecM = VECM(endog=sf, k_ar_diff=lag_order-1, coint_rank=1, deterministic='ci')
    vecm_fit = vecM.fit()
    print(vecm_fit.summary())

# 미국 금리 더 높을 때 

In [65]:
all_test(df["2005-07-05":"2007-08-09"])

              kospi  us_tb_3m
Date                         
2005-07-05  1018.81      3.22
2005-07-06  1019.01      3.19
2005-07-07  1026.82      3.15
2005-07-08  1021.95      3.17
2005-07-11  1040.43      3.19
ADF TEST
ADF Test result of kospi: 0.9656161979162848
ADF Test result of us_tb_3m: 0.09135986365112209
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -1.361     -1.293*      0.2565     -1.334*
1       -1.354      -1.253      0.2583      -1.314
2       -1.346      -1.212      0.2602      -1.294
3       -1.363      -1.195      0.2559      -1.297
4       -1.386      -1.184      0.2502      -1.306
5       -1.416      -1.181      0.2426      -1.324
6       -1.406      -1.137      0.2451      -1.300
7       -1.393      -1.090      0.2484      -1.274
8       -1.405      -1.068      0.2455      -1.273
9       -1.402      -1.032      0.2461      -1.257
10     -1.430* 

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 524

In [ ]:
all_test(df["2017-11-13":"2020-03-02"])

In [39]:
all_test(df["2022-06-30":"2024-06-11"])

              kospi  us_tb_3m
Date                         
2022-06-30  2332.64      1.72
2022-07-01  2305.42      1.73
2022-07-04  2300.34      1.73
2022-07-05  2341.78      1.90
2022-07-06  2292.01      1.90
ADF TEST
ADF TEST for call rate
                  0
ADF Score -2.021738
p-value    0.277124
ADF TEST for us rate
                  0
ADF Score -4.760582
p-value    0.000065
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       0.1996      0.2719       1.221      0.2281
1      0.1439*     0.2524*      1.155*     0.1866*
2       0.1582      0.3029       1.171      0.2152
3       0.1677      0.3485       1.183      0.2390
4       0.1830      0.4000       1.201      0.2685
5       0.1712      0.4244       1.187      0.2710
6       0.1776      0.4669       1.194      0.2916
7       0.1904      0.5159       1.210      0.3186
8       0.1926      0.5542       1.212      0.3

# 글로벌 위기 별

In [40]:
all_test(df["2001-01-02":"2008-09-15"])

             kospi  us_tb_3m
Date                        
2001-01-02  520.95      5.87
2001-01-03  521.43      5.69
2001-01-04  558.02      5.37
2001-01-05  580.85      5.12
2001-01-08  586.65      5.19
ADF TEST
ADF TEST for call rate
                  0
ADF Score -0.899294
p-value    0.788205
ADF TEST for us rate
                  0
ADF Score -1.422863
p-value    0.571267
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0      0.01116     0.03475       1.011     0.01985
1    -0.008471    0.02690*      0.9916    0.004558
2     -0.01383     0.03333      0.9863    0.003545
3     -0.03022     0.02874      0.9702  -0.008502*
4     -0.02842     0.04233      0.9720   -0.002360
5     -0.02678     0.05575      0.9736    0.003620
6     -0.02783     0.06649      0.9726    0.006913
7     -0.02493     0.08118      0.9754     0.01416
8     -0.02502     0.09288      0.9753     0.01840
9  

In [41]:
all_test(df["2008-09-15":"2019-12-31"])

              kospi  us_tb_3m
Date                         
2008-09-16  1387.75      0.84
2008-09-17  1425.26      0.03
2008-09-18  1392.42      0.23
2008-09-19  1455.78      0.99
2008-09-22  1460.34      1.28
ADF TEST
ADF TEST for call rate
                  0
ADF Score -2.007371
p-value    0.283351
ADF TEST for us rate
                  0
ADF Score -0.399441
p-value    0.910192
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -1.942      -1.925      0.1435      -1.935
1       -1.947      -1.921      0.1427      -1.937
2       -1.947      -1.913      0.1426      -1.935
3       -1.959      -1.916      0.1411      -1.943
4       -1.976      -1.924      0.1387      -1.957
5       -1.978      -1.918      0.1383      -1.957
6       -2.003      -1.934      0.1350      -1.978
7       -2.001      -1.924      0.1351      -1.974
8       -2.035     -1.949*      0.1307      -2.

In [42]:
all_test(df["2019-12-31":"2023-05-05"])

              kospi  us_tb_3m
Date                         
2020-01-02  2175.17      1.54
2020-01-03  2176.46      1.52
2020-01-06  2155.07      1.56
2020-01-07  2175.54      1.54
2020-01-08  2151.31      1.54
ADF TEST
ADF TEST for call rate
                  0
ADF Score -1.338511
p-value    0.611375
ADF TEST for us rate
                  0
ADF Score  1.229956
p-value    0.996186
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       0.6377     0.6844*       1.892     0.6557*
1       0.6427      0.7127       1.902      0.6696
2       0.6386      0.7319       1.894      0.6744
3       0.6431      0.7598       1.902      0.6879
4       0.6315      0.7715       1.880      0.6852
5       0.6046      0.7679       1.830      0.6673
6       0.6081      0.7948       1.837      0.6798
7       0.6160      0.8260       1.852      0.6967
8       0.6244      0.8577       1.867      0.7

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 827

In [ ]:
all_test(df["2023-05-05":"2024-06-11"])

In [ ]:
all_test(df["2001-01-02":"2024-06-11"])

## 2005-07-07 ~ 2008-09-09

In [66]:
all_test(df['2005-07-07':'2008-09-09'])

              kospi  us_tb_3m
Date                         
2005-07-07  1026.82      3.15
2005-07-08  1021.95      3.17
2005-07-11  1040.43      3.19
2005-07-12  1043.88      3.21
2005-07-13  1050.16      3.22
ADF TEST
ADF Test result of kospi: 0.32136666441616596
ADF Test result of us_tb_3m: 0.9674084635462563
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        1.184      1.232*       3.266       1.202
1        1.167       1.239       3.211       1.195
2        1.157       1.254       3.181       1.195
3        1.131       1.252       3.098      1.178*
4        1.133       1.279       3.105       1.189
5        1.140       1.310       3.127       1.205
6        1.138       1.333       3.121       1.213
7        1.143       1.362       3.136       1.227
8        1.149       1.391       3.154       1.242
9        1.153       1.420       3.167       1.256
10       1.156 

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 787

## 2009-02-24 ~ 2011-06-10

In [44]:
all_test(df['2009-02-24':'2011-06-10'])

              kospi  us_tb_3m
Date                         
2009-02-24  1063.88      0.32
2009-02-25  1067.08      0.30
2009-02-26  1054.79      0.27
2009-02-27  1063.03      0.26
2009-03-02  1018.81      0.28
ADF TEST
ADF TEST for call rate
                  0
ADF Score -2.221363
p-value    0.198584
ADF TEST for us rate
                  0
ADF Score -2.137884
p-value    0.229556
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -2.983     -2.921*     0.05064     -2.959*
1       -2.973      -2.880     0.05113      -2.937
2      -2.990*      -2.865    0.05031*      -2.941
3       -2.988      -2.832     0.05039      -2.927
4       -2.980      -2.793     0.05079      -2.907
5       -2.972      -2.754     0.05120      -2.887
6       -2.967      -2.718     0.05144      -2.870
7       -2.956      -2.675     0.05204      -2.846
8       -2.957      -2.646     0.05196      -2.

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 577

## 2021-07-06 ~ 2024-06-11

In [45]:
all_test(df['2021-07-06':'2024-06-11'])

              kospi  us_tb_3m
Date                         
2021-07-06  3305.21      0.05
2021-07-07  3285.34      0.05
2021-07-08  3252.68      0.06
2021-07-09  3217.95      0.06
2021-07-12  3246.47      0.05
ADF TEST
ADF TEST for call rate
                  0
ADF Score -2.688669
p-value    0.076020
ADF TEST for us rate
                  0
ADF Score -1.355872
p-value    0.603239
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       0.3353     0.3874*       1.398      0.3554
1      0.3211*      0.3993      1.379*     0.3513*
2       0.3299      0.4342       1.391      0.3702
3       0.3320      0.4623       1.394      0.3824
4       0.3424      0.4988       1.408      0.4029
5       0.3261      0.5086       1.386      0.3967
6       0.3318      0.5403       1.394      0.4124
7       0.3336      0.5681       1.396      0.4243
8       0.3403      0.6009       1.405      0.4

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 721

# 국내 금리 하락

## 2001-01-02 ~ 2005-07-07 

In [46]:
all_test(df['2001-01-02':'2005-07-07'])

             kospi  us_tb_3m
Date                        
2001-01-02  520.95      5.87
2001-01-03  521.43      5.69
2001-01-04  558.02      5.37
2001-01-05  580.85      5.12
2001-01-08  586.65      5.19
ADF TEST
ADF TEST for call rate
                  0
ADF Score -1.191604
p-value    0.677197
ADF TEST for us rate
                  0
ADF Score -3.508819
p-value    0.007763
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -1.674     -1.638*      0.1874      -1.660
1       -1.682      -1.627      0.1861     -1.661*
2       -1.680      -1.607      0.1864      -1.652
3       -1.676      -1.584      0.1871      -1.641
4      -1.685*      -1.575     0.1855*      -1.643
5       -1.678      -1.549      0.1868      -1.629
6       -1.671      -1.525      0.1880      -1.616
7       -1.683      -1.518      0.1859      -1.620
8       -1.675      -1.492      0.1872      -1.606
9  

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 1113

## 2008-09-09 ~ 2009-02-24

In [47]:
all_test(df['2008-09-09':'2009-02-24'])

              kospi  us_tb_3m
Date                         
2008-09-09  1454.50      1.66
2008-09-10  1464.98      1.65
2008-09-11  1443.24      1.61
2008-09-12  1477.92      1.49
2008-09-16  1387.75      0.84
ADF TEST
ADF TEST for call rate
                  0
ADF Score -2.033111
p-value    0.272246
ADF TEST for us rate
                  0
ADF Score -2.673598
p-value    0.078719
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        2.236       2.457       9.359       2.325
1        2.079      2.410*       8.000       2.213
2        2.042       2.484       7.713       2.220
3        2.053       2.605       7.808       2.276
4        2.076       2.738       7.996       2.343
5        2.109       2.881       8.279       2.420
6        1.997       2.880       7.420       2.353
7        1.885       2.878       6.658       2.286
8        1.795       2.898       6.106       2.

## 2011-06-10 ~ 2021-07-06

In [48]:
all_test(df['2011-06-10':'2021-07-06'])

              kospi  us_tb_3m
Date                         
2011-06-10  2046.67      0.05
2011-06-13  2048.74      0.05
2011-06-14  2076.83      0.05
2011-06-15  2086.53      0.05
2011-06-16  2046.63      0.05
ADF TEST
ADF TEST for call rate
                  0
ADF Score  0.210784
p-value    0.972860
ADF TEST for us rate
                  0
ADF Score -1.055541
p-value    0.732472
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -1.724      -1.705      0.1783      -1.717
1       -1.737      -1.708      0.1761      -1.726
2       -1.743      -1.706      0.1749      -1.730
3       -1.756      -1.709      0.1728      -1.739
4       -1.766      -1.709      0.1710      -1.745
5       -1.780     -1.714*      0.1687     -1.756*
6       -1.783      -1.707      0.1682      -1.755
7       -1.782      -1.697      0.1683      -1.751
8       -1.780      -1.685      0.1687      -1.

# 미국 국채 금리 상승 

## 2003-06-19 ~ 2007-02-22

In [49]:
all_test(df['2003-06-19':'2007-02-22'])

             kospi  us_tb_3m
Date                        
2003-06-19  690.49      0.81
2003-06-20  686.22      0.83
2003-06-23  674.59      0.85
2003-06-24  664.01      0.83
2003-06-25  674.03      0.92
ADF TEST
ADF TEST for call rate
                  0
ADF Score -0.531967
p-value    0.885626
ADF TEST for us rate
                  0
ADF Score -0.015306
p-value    0.957248
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -2.200     -2.157*      0.1108      -2.183
1       -2.200      -2.135      0.1108      -2.175
2       -2.203      -2.117      0.1105      -2.170
3       -2.222      -2.115      0.1084      -2.181
4       -2.236      -2.107      0.1069      -2.187
5       -2.294      -2.143      0.1009     -2.236*
6       -2.288      -2.116      0.1014      -2.222
7       -2.281      -2.087      0.1022      -2.207
8       -2.293      -2.078      0.1009      -2.211
9  

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 914

##  2008-11-28 ~ 2019-03-21

In [50]:
all_test(df['2008-11-28':'2019-03-21'])

              kospi  us_tb_3m
Date                         
2008-11-28  1076.07      0.01
2008-12-01  1058.62      0.07
2008-12-02  1023.20      0.06
2008-12-03  1022.67      0.02
2008-12-04  1006.54      0.02
ADF TEST
ADF TEST for call rate
                  0
ADF Score -3.055354
p-value    0.030038
ADF TEST for us rate
                  0
ADF Score  4.491083
p-value    1.000000
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -2.477     -2.458*     0.08403      -2.470
1       -2.481      -2.453     0.08369      -2.471
2       -2.493      -2.456     0.08262      -2.480
3       -2.497      -2.451     0.08230     -2.481*
4       -2.496      -2.440     0.08244      -2.476
5       -2.497      -2.433     0.08231      -2.474
6       -2.498      -2.424     0.08226      -2.471
7       -2.498      -2.414     0.08228      -2.467
8       -2.497      -2.405     0.08231      -2.

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 2548

## 2020-03-26 ~ 2024-06-11

In [ ]:
all_test(df['2020-03-26':'2024-06-11'])

## 미국 금리 하락 

## 2001-01-02 ~ 2003-06-19

In [51]:
all_test(df['2001-01-02':'2003-06-19'])

             kospi  us_tb_3m
Date                        
2001-01-02  520.95      5.87
2001-01-03  521.43      5.69
2001-01-04  558.02      5.37
2001-01-05  580.85      5.12
2001-01-08  586.65      5.19
ADF TEST
ADF TEST for call rate
                  0
ADF Score -1.781668
p-value    0.389600
ADF TEST for us rate
                  0
ADF Score -3.119621
p-value    0.025135
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -1.133     -1.073*      0.3220      -1.110
1      -1.150*      -1.060     0.3166*     -1.115*
2       -1.141      -1.021      0.3195      -1.094
3       -1.130     -0.9806      0.3229      -1.072
4       -1.136     -0.9564      0.3210      -1.066
5       -1.135     -0.9247      0.3216      -1.053
6       -1.121     -0.8815      0.3259      -1.028
7       -1.124     -0.8538      0.3251      -1.018
8       -1.112     -0.8127      0.3288     -0.9956
9  

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 606

## 2007-02-22 ~ 2008-11-28

In [52]:
all_test(df['2007-02-22 ':'2008-11-28'])

              kospi  us_tb_3m
Date                         
2007-02-22  1465.41      5.19
2007-02-23  1469.88      5.18
2007-02-26  1470.03      5.19
2007-02-27  1454.60      5.14
2007-02-28  1417.34      5.16
ADF TEST
ADF TEST for call rate
                  0
ADF Score -0.303705
p-value    0.924978
ADF TEST for us rate
                  0
ADF Score -0.215658
p-value    0.936629
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        2.923      3.001*       18.60       2.954
1        2.905       3.022       18.26       2.951
2        2.886       3.041       17.92       2.947
3       2.851*       3.046      17.31*      2.928*
4        2.866       3.099       17.56       2.958
5        2.871       3.143       17.65       2.979
6        2.862       3.173       17.50       2.985
7        2.874       3.224       17.71       3.012
8        2.882       3.271       17.85       3.

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 437

##  2019-03-21 ~ 2020-03-26

In [53]:
all_test(df['2019-03-21':'2020-03-26'])

              kospi  us_tb_3m
Date                         
2019-03-21  2184.88      2.49
2019-03-22  2186.95      2.46
2019-03-25  2144.86      2.46
2019-03-26  2148.80      2.46
2019-03-27  2145.62      2.44
ADF TEST
ADF TEST for call rate
                  0
ADF Score -2.099078
p-value    0.244885
ADF TEST for us rate
                  0
ADF Score  1.230435
p-value    0.996190
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       0.1694      0.2890       1.185      0.2176
1      0.09113      0.2705       1.095      0.1635
2     0.002882     0.2421*       1.003     0.09936
3     -0.04597      0.2530      0.9552     0.07463
4     -0.07953      0.2792      0.9237    0.06519*
5     -0.06247      0.3561      0.9397      0.1064
6     -0.05832      0.4200      0.9438      0.1346
7     -0.04990      0.4882      0.9519      0.1672
8     -0.05297      0.5450      0.9492      0.1

# 한국 금리가 높을 떄

## 2001-02-26 ~ 2005-07-05

In [54]:
all_test(df['2001-02-26':'2005-07-05'])

             kospi  us_tb_3m
Date                        
2001-02-26  585.32      4.81
2001-02-27  577.57      4.82
2001-02-28  578.10      4.85
2001-03-02  559.44      4.84
2001-03-05  565.38      4.82
ADF TEST
ADF TEST for call rate
                  0
ADF Score -1.013468
p-value    0.748321
ADF TEST for us rate
                  0
ADF Score -2.545780
p-value    0.104729
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0       -1.691     -1.653*      0.1844     -1.676*
1       -1.697      -1.640      0.1832      -1.676
2       -1.695      -1.619      0.1837      -1.666
3       -1.690      -1.595      0.1846      -1.654
4       -1.703      -1.590      0.1821      -1.661
5       -1.698      -1.566      0.1831      -1.648
6       -1.691      -1.540      0.1844      -1.634
7      -1.705*      -1.536     0.1817*      -1.641
8       -1.698      -1.509      0.1831      -1.626
9  

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 1075

## 2007-08-09 ~ 2017-11-13

In [55]:
all_test(df['2007-08-09':'2017-11-13'])

              kospi  us_tb_3m
Date                         
2007-08-09  1908.68      4.81
2007-08-10  1828.49      4.57
2007-08-13  1849.26      4.74
2007-08-14  1817.89      4.64
2007-08-16  1691.98      3.79
ADF TEST
ADF TEST for call rate
                  0
ADF Score -1.237581
p-value    0.657224
ADF TEST for us rate
                  0
ADF Score -5.077811
p-value    0.000016
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0      0.02360     0.04213       1.024     0.03033
1     -0.01858    0.009215      0.9816   -0.008495
2     -0.09787    -0.06081      0.9068    -0.08442
3      -0.1086   -0.06231*      0.8971    -0.09183
4      -0.1114    -0.05579      0.8946    -0.09121
5      -0.1127    -0.04786      0.8934    -0.08919
6      -0.1235    -0.04936      0.8838    -0.09659
7      -0.1316    -0.04825      0.8766     -0.1014
8      -0.1508    -0.05813      0.8600    -0.11

## 2020-03-02 ~ 2022-06-30

In [56]:
all_test(df['2020-03-02':'2022-06-30'])

              kospi  us_tb_3m
Date                         
2020-03-02  2002.51      1.13
2020-03-03  2014.15      0.95
2020-03-04  2059.33      0.72
2020-03-05  2085.26      0.62
2020-03-06  2040.22      0.45
ADF TEST
ADF TEST for call rate
                  0
ADF Score -1.405119
p-value    0.579820
ADF TEST for us rate
                 0
ADF Score  7.58912
p-value    1.00000
Find Lag
 VECM Order Selection (* highlights the minimums) 
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0      -0.4490    -0.3867*      0.6383     -0.4246
1      -0.4384     -0.3449      0.6451     -0.4019
2      -0.4410     -0.3163      0.6434     -0.3923
3     -0.4882*     -0.3323     0.6138*    -0.4273*
4      -0.4761     -0.2891      0.6212     -0.4030
5      -0.4674     -0.2492      0.6266     -0.3822
6      -0.4627     -0.2134      0.6296     -0.3653
7      -0.4640     -0.1834      0.6288     -0.3544
8      -0.4630     -0.1513      0.6294     -0.3412

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 577